In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from raagam_player import RaagPlayer



try: from ipywidgets import Dropdown
except: 
    !pip install ipywidgets
    from ipywidgets import Dropdown


In [8]:
mela_df = pd.read_excel(r"All_Raagams.xlsx")
mela_df.columns = ["_".join(i.strip().split()) for i in mela_df.columns]

for n in ['AAROHANAM','AVAROHANAM']:
    mela_df[n]  = mela_df[n].apply(lambda x: str(x).replace("\n","").split())
mela_df[mela_df['Raga_Name'].isnull()==False]
mela_df.head()

,Raga_Name_Raw,Raga_Name,AAROHANAM,AVAROHANAM,Melakarta,Melakarta_Name
0,1 Kanakāngi (Janaka raga),Kanakāngi (Janaka raga),"[S, R1, G1, M1, P, D1, N1, S]","[S, N1, D1, P, M1, G1, R1, S]",1.0,Kanakāngi (Janaka raga)
1,Kanakāmbari,Kanakāmbari,"[S, R1, M1, P, D1, S]","[S, N1, D1, P, M1, G1, R1, S]",0.0,Kanakāngi (Janaka raga)
2,Kanakatodi,Kanakatodi,"[S, R1, G1, M1, P, D1, S]","[S, N1, D1, P, M1, R1, S]",0.0,Kanakāngi (Janaka raga)
3,Mādhavapriyā,Mādhavapriyā,"[S, R1, G1, P, D1, N1, S]","[S, N1, D1, P, G1, R1, S]",0.0,Kanakāngi (Janaka raga)
4,Karnātaka Shuddha Sāveri,Karnātaka Shuddha Sāveri,"[S, R1, M1, P, D1, S]","[S, D1, P, M1, R1, S]",0.0,Kanakāngi (Janaka raga)


In [70]:
raag_notes = RaagPlayer().notes_list
raag_tuple = [(i+1,n,v) for i,(n,v) in enumerate(raag_notes.items())]


line = [(x, (1/17)*x+1)  for x in range(1,18)]
non3 = [i for i in raag_tuple if '3' not in i[1]]
# # px.scatter(pd.DataFrame(RaagPlayer().notes_list, index = [0]).T,  trendline = "ols")
plt.figure(figsize = (10,10))
px.scatter(x = [i[0] for i in non3],
           y = [i[-1] for i in non3], 
           trendline='ols')
px.scatter(x = [i[0] for i in line] + [i[0] for i in non3],
           y = [i[1] for i in line]+ [i[-1] for i in non3])




<Figure size 1000x1000 with 0 Axes>

In [21]:
raag_i = np.random.choice(range(mela_df.shape[0]),size=1)[0]
raag = mela_df.iloc[raag_i].to_dict()
print(raag['Raga_Name'])
# RaagPlayer(list(raag['AAROHANAM'])[:-1])
RaagPlayer([raag['AAROHANAM'], raag['AVAROHANAM']])


Gamakapriyā
[['S', 'R1', 'G3', 'M2', 'P', 'N3', 'D1', 'S'], ['S', 'D1', 'P', 'M2', 'G3', 'R1', 'S']]
S  R1 G3 M2 P  N3 D1 S2 
 S2 D1 P  M2 G3 R1 S  

In [20]:
mela_df = mela_df[mela_df['Raga_Name']!=0]
d = Dropdown(options = sorted(mela_df['Raga_Name']))

def print_selected(change):
  selected_value = change['new']  
  print(f"You selected: {selected_value}")
  RaagPlayer(mela_df[mela_df['Raga_Name']==selected_value]['AAROHANAM'].tolist()[0][:-1])
  print("\n")
d.observe(print_selected, names='value')
display(d)

Dropdown(options=('Abheri', 'Abheri (Dikshitar School)', 'Abhiru', 'Abhirāmam', 'Abhogi', 'Agnikopa', 'Alankār…

You selected: Harikāmbhōji
S  R2 G3 M1 P  D2 N2 S  

TypeError: can't multiply sequence by non-int of type 'float'

In [12]:
mela_df['Raga_Name']

0        Kanakāngi (Janaka raga)
1                    Kanakāmbari
2                     Kanakatodi
3                   Mādhavapriyā
4       Karnātaka Shuddha Sāveri
                  ...           
1007                           0
1008                           0
1009                           0
1010                           0
1011                           0
Name: Raga_Name, Length: 1012, dtype: object